<a href="https://www.kaggle.com/code/davidhalim2004/data-cleaning?scriptVersionId=280113860" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Library importing

In [1]:
!pip install xgboost
import pandas as pd
import joblib
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


# data pre-proccessing

## data reading

In [2]:
chunks = pd.read_csv("/kaggle/input/data.csv", chunksize=5_000_000)
final_data = pd.concat(chunks, ignore_index=True)

In [19]:
del chunks

In [20]:
preprocessor=joblib.load("/kaggle/input/pre-proccess.pkl")

## Data spliting

In [21]:
y=final_data['sales']
x=final_data.drop(['sales'],axis = 1)

# Model

In [22]:
del final_data
import gc
gc.collect()

0

In [23]:
categorical_cols = ['store_id', 'state_id', 'item_category', 'item_subcategory']
numeric_cols = [ "wday", "snap","year", "month", "sell_price","lag_1",'lag_7','snap_weekend','wday_x_snap','is_weekend']
for col in categorical_cols:
    if x[col].dtype == "object":
        x[col] = x[col].astype("category")
for col in numeric_cols:
    x[col] = pd.to_numeric(x[col], downcast="float")
del categorical_cols
del numeric_cols
gc.collect()

0

In [24]:
del col

In [25]:
gc.collect()

0

In [26]:
x=preprocessor.fit_transform(x)

ValueError: A given column is not a column of the dataframe

In [ ]:
model = XGBRegressor(
    objective='reg:tweedie',
    tweedie_variance_power=1.35,
    booster='gbtree',
    device='cuda',
    tree_method='hist',
    random_state=42,
    eval_metric='mae',
    learning_rate=0.03,
    max_depth=10,
    min_child_weight=8,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=6,
    reg_alpha=2,
    n_estimators=3000,
    early_stopping_rounds=40,
    gamma=0.1,
    max_delta_step = 1,
)

In [ ]:
x_temp , x_test , y_temp , y_test = train_test_split(x , y , test_size = 0.20 , random_state = 42)
del x,y
gc.collect()
x_train , x_val , y_train , y_val = train_test_split(x_temp , y_temp , test_size = 0.15 , random_state = 42)
del x_temp,y_temp
gc.collect()

In [ ]:
gc.collect()

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score

train_mae = mean_absolute_error(y_train, model.predict(x_train))
test_mae  = mean_absolute_error(y_test,  model.predict(x_test))
train_r2  = r2_score(y_train, model.predict(x_train))
test_r2   = r2_score(y_test,  model.predict(x_test))

print(f"MAE train: {train_mae:.4f}, test: {test_mae:.4f}")
print(f"R2  train: {train_r2:.4f}, test: {test_r2:.4f}")

In [ ]:
model.fit(x_train,y_train,eval_set = [(x_val , y_val)])

In [ ]:
model.score(x_train,y_train) , model.score(x_test,y_test)

In [ ]:
import numpy as np
y_predict=model.predict(x_test)
y_pred_rounded = np.round(y_predict).astype(int)
y_test_rounded = np.round(y_test).astype(int)
ct=0
for i in range (0,25):
    print(y_pred_rounded[i], y_test_rounded.iloc[i])